In [ ]:
# -*- coding: utf-8 -*-
import recon
import plotting as plot
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import utilities
import warnings
warnings.filterwarnings('ignore')


model = recon.Recon()
util = utilities.Utilities()

sns.set_style('white')
sns.set_style('ticks')

## UC analyses

In [ ]:
# Proteomics data
proteomics = pd.read_csv('../data/proteomics_uc_vs_healthy.csv', index_col='number')
proteomics.index = proteomics.index.astype('str')

# Predicted centrality
pr = pd.read_csv('../data/pr_centrality_proteomics_158p.csv', sep='\t', index_col=0)
pr['vmh'] = [i[:-3].lower() for i in pr.index]
pr['mets'] = [i for i in pr.index]

# Metabolomics data
metabolomics = pd.read_csv('../data/metabolomics_uc_vs_healthy.csv', sep=',')
metabolomics.vmh = metabolomics.vmh.str.lower()
for col in metabolomics.columns.difference(['vmh', ]):
    metabolomics.loc[:, col] = metabolomics.loc[:, col].astype(float)
metabolomics.head()

In [ ]:
len(metabolomics)
# p_crit = metabolomics.p < .05
# fc_crit = (metabolomics.fc > 1.2) | (metabolomics.fc < .8) 
# metabolomics = metabolomics[p_crit & fc_crit]
# len(metabolomics)

In [ ]:
cc = util.calculate_cc(g_df=proteomics)
cc_m = util.calculate_cc(g_df=proteomics, grouping='mitochondrial')
cc_c = util.calculate_cc(g_df=proteomics, grouping='cytoplasm')

In [ ]:
sns.set_context("talk")

plot.hist(pr=np.log2(pr.mean(axis=1)), 
          save=True, filename='../../images/UC/histplot_uc_patients_proteomics_1.svg', 
          figsize=(8,7), xlim=(-1., 1.), bins=22, color='C1', ylim=(0, 2e4),
          xlabel='log$_2$(predicted change)', ylabel='no. of metabolites')

In [ ]:
def map_metabolomics(metabolomics, pr):
    print(f'mapped: {len(set([i.lower() for i in metabolomics.vmh]) & set([i.lower() for i in pr.vmh]))}')
    print(f'missing: {len(set([i.lower() for i in metabolomics.vmh]) - set([i.lower() for i in pr.vmh]))}')
    missing = list(set([i.lower() for i in metabolomics.vmh]) - set([i.lower() for i in pr.vmh]))
    print(f'\t{missing}')
    m1 = pr.merge(metabolomics, left_on='vmh', right_on='vmh').set_index('mets')
    m2 = util.reshape(m1, include=None)
    m2 = m2.rename(columns={'0': 'prediction_proteomics', 0: 'prediction_proteomics', 'fc': 'metabolomics'})
    m2['p'] = m2['p'].astype('float')
    m2['metabolomics'] = m2['metabolomics'].astype('float')
    m2['prediction_proteomics'] = m2['prediction_proteomics'].astype('float')
    return m2

In [ ]:
met = metabolomics.copy()
m1 = map_metabolomics(metabolomics=met, pr=pr)
p_crit = met.p < .05
fc_crit = (met.fc > 1.2) | (met.fc < .8)
metabolomics2 = met[p_crit & fc_crit]
m2 = map_metabolomics(metabolomics=metabolomics2, pr=pr)

In [ ]:
# m2.groupby('metabolites').mean()
# m1 = m1[m1.p < .05]
# df = pd.concat((m1[m1.compartment == 'cytoplasm'].groupby('metabolites').mean(), cc), axis=1).dropna()
# df = pd.concat((m1[m1.compartment == 'mitochondrial'].groupby('metabolites').mean(), cc_m), axis=1).dropna()
# df = pd.concat((m1[m1.compartment.isin(['mitochondrial', 'cytoplasm', 'nuclear'])].groupby('metabolites').mean(), cc), axis=1).dropna()
df = pd.concat((m1[m1.compartment != 'extracellular'].groupby('metabolites').mean(numeric_only=True), cc), axis=1).dropna()
df = df.rename(columns={0: 'control_coeff'})
# df['hue'] = 'C1'
# df.loc[(df.p > 0.05), 'hue'] = 'grey'
# df.loc[(df.prediction_proteomics < 1.01) & (df.prediction_proteomics > 0.99), 'hue'] = 'grey'
# df.loc[(df.metabolomics < 1.2) & (df.metabolomics > 0.8), 'hue'] = 'grey'

import utility
coverage = {}
for met in df.index:
    try:
        coverage[met] = 100*len(proteomics.index.intersection(utility.get_genes_from_metabolite(model, met)))/len(utility.get_genes_from_metabolite(model, met))
    except ZeroDivisionError:
        coverage[met] = 0.0
df['coverage (%)'] = pd.Series(coverage)

In [ ]:
df.to_csv('../data/Supplementary_Table_S2.csv', sep='\t', float_format='%.3f')

In [ ]:
df = df.reset_index()
df.columns = ['Metabolite name', 'Predicted', 'Measured', 'p', 'Control coeff', 'Coverage (%)']

In [ ]:
df.to_csv('../data/Supplementary_Table_S2.csv', sep='\t', float_format='%.3f')

In [ ]:
sns.set_context('talk')
fig, ax = plt.subplots(figsize=(8,7))
ax.scatter(x=np.log2(df[df.hue=='grey']['prediction_proteomics']), y=np.log2(df[df.hue=='grey']['metabolomics']), 
           c='grey', alpha=0.3)
ax.scatter(x=np.log2(df[df.hue!='grey']['prediction_proteomics']), y=np.log2(df[df.hue!='grey']['metabolomics']), 
           c='C1', alpha=0.8)
ax.axhline(y=0.0, c='k', ls='--')
ax.axvline(x=0.0, c='k', ls='--')
plt.xlim(-0.25, 0.25)
plt.ylim(-3.5, 3.5)
ax.set_xlabel('log$_2$(predicted change)')
ax.set_ylabel('log$_2$(measured change)')

# plot.add_percentage(df=df, ax=ax, colname='prediction_proteomics', colname2='metabolomics',
#                     show_sc_percentages=False, fsize=15,
#                     l=(0.1, 0.9), l2=(0.7, 0.9), l3=(0.1, 0.3), l4=(0.7, 0.3))
plot.add_percentage(df=df[df.hue!='grey'], ax=ax, colname2='prediction_proteomics', colname='metabolomics',
                    show_sc_percentages=False, fsize=15,
                    l=(0.02, 0.95), l2=(0.52, 0.95), l3=(0.02, 0.25), l4=(0.52, 0.25))
plot.add_metabolite_names1(df=df[df.hue!='grey'], ax=ax, colname='prediction_proteomics', colname2='metabolomics', 
                           fsize=10, l=(0.15, 0.95), l2=(0.65, 0.95), l3=(0.15, 0.25), l4=(0.65, 0.25),
                           )
fig.savefig('../../images/UC/UC_metabolomics_vs_proteomics_1.png', bbox_inches='tight', dpi=300)
fig.savefig('../../images/UC/UC_metabolomics_vs_proteomics_1.svg', bbox_inches='tight')


In [ ]:
_df = df.loc[(df.p < 0.05)].drop(['p', 'control_coeff', 'hue'], axis=1).rename(columns={'metabolomics':'measured change', 'prediction_proteomics':'predicted change'})
_df.loc[(_df['predicted change'] > 1.0) & (_df['measured change'] > 1.0), 'quadrant'] = 'UR'
_df.loc[(_df['predicted change'] < 1.0) & (_df['measured change'] > 1.0), 'quadrant'] = 'UL'
_df.loc[(_df['predicted change'] < 1.0) & (_df['measured change'] < 1.0), 'quadrant'] = 'LL'
_df.loc[(_df['predicted change'] > 1.0) & (_df['measured change'] < 1.0), 'quadrant'] = 'LR'
_df #= _df.merge(mets, left_on=_df.index, right_on='metabolite').set_index('metabolite')
# _df#.reset_index().to_excel('../manuscript/results_uc_patients.xlsx', index=False)

In [ ]:
_coverage = pd.read_excel('../../manuscript/results_uc_patients.xlsx')

In [ ]:
model.gs.to_excel('../../manuscript/control_coefficients.xlsx')

In [ ]:
sns.scatterplot(data=_coverage, x='number of rxns with gpr', y='number of rxns with mapped gpr', hue='quadrant')
# _coverage.columns

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')
# df = pd.concat((m2.groupby('metabolites').mean(), cc), axis=1).dropna()
# df = df.rename(columns={0: 'control_coeff'})
_df = df.copy()
# df = df[~df.index.str.contains("arnitine")]
_df = df.loc[(df.p < 0.05)]
_df = _df[(_df['metabolomics'] > 1.2) | (_df['metabolomics'] < 0.8)]
_df = _df[(_df['prediction_proteomics'] > 1.01) | (_df['prediction_proteomics'] < 0.99)]

plot.parity(data=_df, xcolumn='metabolomics', ycolumn='prediction_proteomics', cc=False, 
            figsize=(10, 8), save=True, filename='../../images/UC/parity_UC_patients_proteomics_158p.png',
            ylabel='log$_2$(measured change)', xlabel='log$_2$(predicted change)', color='C1',
            edgecolor=None,
            show_percentage=True, show_names=True, ub=0.8, lb=0.0, nb=5, cbar_cmap='Oranges',
            n=[1, 1, 1, 1], fsize=9, ylim=(-2.5, 2.5), xlim=(-0.25, 0.25), alpha=1.0,
            l_p=[(0.05, 0.95), (0.55, 0.95), (0.05, 0.25), (0.55, 0.25)],
            l_n=[(0.05, 0.9), (0.65, 0.95), (0.05, 0.2), (0.65, 0.25)]
            )

plot.parity(data=_df, xcolumn='metabolomics', ycolumn='prediction_proteomics', cc=True, cc_column='control_coeff', 
            figsize=(8, 7), save=True, filename='../../images/UC/parity_UC_patients_proteomics_158p_probability.png',
            ylabel='log$_2$(measured change)', xlabel='log$_2$(predicted change)', color='C1',
            show_percentage=True, show_names=True, ub=0.5, lb=0.0, nb=6, cbar_cmap='Reds',
            n=[1, 1, 1, 1], fsize=9, ylim=(-3.5, 3.5), xlim=(-0.25, 0.25), alpha=1.0,
            l_p=[(0.05, 0.95), (0.55, 0.95), (0.05, 0.25), (0.55, 0.25)],
            l_n=[(0.05, 0.9), (0.65, 0.95), (0.05, 0.2), (0.65, 0.25)]
            )

plot.parity(data=_df, xcolumn='metabolomics', ycolumn='prediction_proteomics', 
            cc=True, cc_column='coverage', cbar_label='coverage of metabolic genes by proteomics',
            figsize=(8, 6), save=True, filename='../../images/UC/parity_UC_patients_proteomics_158p_coverageProteomics.png',
            ylabel='log$_2$(measured change)', xlabel='log$_2$(predicted change)', color='C1',
            show_percentage=True, show_names=True, ub=0.8, lb=0.0, nb=5, cbar_cmap='Blues',
            n=[1, 1, 1, 1], fsize=9, ylim=(-3.5, 3.5), xlim=(-0.25, 0.25), alpha=1.0,
            l_p=[(0.05, 0.95), (0.55, 0.95), (0.05, 0.25), (0.55, 0.25)],
            l_n=[(0.05, 0.9), (0.65, 0.95), (0.05, 0.2), (0.65, 0.25)]
            )

In [ ]:
sns.set_context('talk')
for comp in m1.compartment.unique():
    df = pd.concat((m1[m1.compartment==comp].groupby('metabolites').mean(), cc), axis=1).dropna()
    df = df.rename(columns={0: 'control_coeff'})
    _df = df.loc[(df.p < 0.05)]
    _df = _df[(_df['metabolomics'] > 1.2) | (_df['metabolomics'] < 0.8)]
    _df = _df[(_df['prediction_proteomics'] > 1.02) | (_df['prediction_proteomics'] < 0.98)]
    if len(_df) > 0:
        if comp == 'cytoplasm':
            ftitle = 'cytoplasmic'
        else:
            ftitle = f'{comp}'
        plot.parity(data=_df, xcolumn='metabolomics', ycolumn='prediction_proteomics', cc=False, 
                figsize=(8, 6), save=True, filename=f'../images/UC/UC_metabolomics_vs_proteomics_{comp}.png',
                show_percentage=True, show_names=True, ub=0.4, lb=0.0,
                n=[1, 1, 1, 1], fsize=10, ylim=(-2.5, 2.5), xlim=(-0.25, 0.25),
                l_p=[(0.05, 0.95), (0.62, 0.95), (0.05, 0.25), (0.68, 0.25)],
                l_n=[(0.05, 0.9), (0.62, 0.9), (0.05, 0.2), (0.68, 0.2)],
                figtitle=ftitle)
    else:
        print(f'skipping {comp} as no metabolites are significant')
        print(f'{comp}: {len(_df)}')
   